In [1]:
import os
import numpy as np
from astropy.table import Table, join
import matplotlib.pyplot as plt
from astropy import units as u
import sys
import edge_pydb
from edge_pydb import xy2hist
from edge_pydb.edge_conv import sfr_ha, msd_co
np.seterr(invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Spatially resolved SF law for EDGE.  

## x-axis: 7" CO intensity from the smoothed mask.  

## y-axis: Halpha flux from flux_elines smoothed to 7".

In [2]:
try: 
    edge_pydb.getfiles('edge.pipe3d.hdf5')
    print('Working on full EDGE database')
    hafile = edge_pydb.getfiles('edge.pipe3d.hdf5')
    cofile = edge_pydb.getfiles('edge.comom_smo7.hdf5')
except:
    print('Working on NGC 4047 data')
    hafile = edge_pydb.getfiles('NGC4047.pipe3d.hdf5')
    cofile = edge_pydb.getfiles('NGC4047.comom_smo7.hdf5')

Working on NGC 4047 data


In [3]:
cotab=Table.read(cofile, path='smo')
hatab=Table.read(hafile, path='flux_elines_sm')
jointbl = join(cotab,hatab)
print(jointbl.colnames)

['Name', 'ix', 'iy', 'ra_off', 'dec_off', 'rad_arc', 'azi_ang', 'emom0max', 'mom0', 'emom0', 'flux_[OII]3727', 'flux_[OIII]5007', 'flux_[OIII]4959', 'flux_Hbeta', 'flux_Halpha', 'flux_[NII]6583', 'flux_[NII]6548', 'flux_[SII]6717', 'flux_[SII]6731', 'vel_[OII]3727', 'vel_[OIII]5007', 'vel_[OIII]4959', 'vel_Hbeta', 'vel_Halpha', 'vel_[NII]6583', 'vel_[NII]6548', 'vel_[SII]6717', 'vel_[SII]6731', 'disp_[OII]3727', 'disp_[OIII]5007', 'disp_[OIII]4959', 'disp_Hbeta', 'disp_Halpha', 'disp_[NII]6583', 'disp_[NII]6548', 'disp_[SII]6717', 'disp_[SII]6731', 'EW_[OII]3727', 'EW_[OIII]5007', 'EW_[OIII]4959', 'EW_Hbeta', 'EW_Halpha', 'EW_[NII]6583', 'EW_[NII]6548', 'EW_[SII]6717', 'EW_[SII]6731', 'e_flux_[OII]3727', 'e_flux_[OIII]5007', 'e_flux_[OIII]4959', 'e_flux_Hbeta', 'e_flux_Halpha', 'e_flux_[NII]6583', 'e_flux_[NII]6548', 'e_flux_[SII]6717', 'e_flux_[SII]6731', 'e_vel_[OII]3727', 'e_vel_[OIII]5007', 'e_vel_[OIII]4959', 'e_vel_Hbeta', 'e_vel_Halpha', 'e_vel_[NII]6583', 'e_vel_[NII]6548', 'e_

In [4]:
# Apply a 3-sigma cut to both axes
det = (jointbl['mom0']>3*jointbl['emom0']) & (
    jointbl['flux_Halpha']>3*jointbl['e_flux_Halpha'])
codata = jointbl['mom0'][det]
print(codata)
hadata = jointbl['flux_Halpha'][det]
sig_mol = msd_co(codata, name='sig_mol')
sig_sfr = sfr_ha(hadata, jointbl['flux_Hbeta'][det], name='sig_sfr')
print('Number of CO and Ha detections:',len(codata))

   mom0  
 K km / s
---------
 7.301712
 8.736801
 7.041168
 8.089447
15.900345
18.578796
15.325394
 9.583817
13.223038
22.714005
      ...
6.2602997
 8.096771
 9.503708
 9.195328
6.9348907
 6.030426
 5.122986
4.6809387
 5.268097
4.5809937
Length = 102 rows
Number of CO and Ha detections: 102


In [5]:
# Make the scatter plot (measured values)
x_det, y_det, z_det, *hist = xy2hist(codata, hadata, log=True)
fig, ax = plt.subplots(figsize=(8,8))
cm = plt.cm.get_cmap('jet')
sc = ax.scatter(x_det, y_det, c=z_det, s=20, edgecolor='', cmap=cm)
plt.xlabel('CO intensity ['+str(codata.unit)+']',fontsize=14)
plt.ylabel(r'H$\alpha$ intensity ['+str(hadata.unit)+']',fontsize=14)
ax.set_aspect('equal')
plt.colorbar(sc)

TypeError: 'module' object is not callable

In [ ]:
not_nan = (~np.isnan(sig_mol)) & (~np.isnan(sig_sfr)) & (sig_mol>0) & (sig_sfr>0)
print('Excluding {} invalid values'.format(len(np.nonzero(~not_nan)[0])))
print('Number of valid points:',len(np.nonzero(not_nan)[0]))

In [ ]:
# Make the scatter plot (physical values)
x_det, y_det, z_det, *hist = xy2hist(sig_mol[not_nan], sig_sfr[not_nan], log=True)
fig, ax = plt.subplots(figsize=(8,8))
cm = plt.cm.get_cmap('jet')
sc = ax.scatter(x_det, y_det, c=z_det, s=20, edgecolor='', cmap=cm)
plt.xlabel(r'$\Sigma_{mol}$ ['+str(sig_mol.unit)+']',fontsize=14)
plt.ylabel(r'$\Sigma_{SFR}$ ['+str(sig_sfr.unit)+']',fontsize=14)
ax.set_aspect('equal')
plt.colorbar(sc)